In [4]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os
import glob
from scipy.misc import imread


from keras.preprocessing import image
from keras.models import load_model

import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

Using TensorFlow backend.


In [1]:
DATA_PATH = '/a/data/fisheries_monitoring/data/'
INPUT_WIDTH = 224
INPUT_HEIGHT = 224

In [2]:
def load_test_data(INPUT_WIDTH, INPUT_HEIGHT):
    X_test_raw = []
    X_test = []
    id_test = []
    path = glob.glob(DATA_PATH + 'test_stg2/*.jpg')
    for i in xrange(len(path)):
        img_raw = image.load_img(path[i])
        img = img_raw
        img_raw = image.img_to_array(img)
        img_raw /= 255

        img = img.resize((INPUT_WIDTH, INPUT_HEIGHT))
        img = image.img_to_array(img)
        img /= 255

        X_test_raw.append(img_raw)
        X_test.append(img)        
        id_test.append(os.path.basename(path[i]))
        
        if (i+1) in [k*len(path)/5 for k in xrange(1,6)]:
                print "Loading...{}% done!".format((i+2)*100/len(path))
    return X_test_raw, np.array(X_test), np.array(id_test)


def test_data_generator(batch_size, files, INPUT_WIDTH, INPUT_HEIGHT):
    while True:
        img_batch = np.zeros((batch_size, INPUT_WIDTH, INPUT_HEIGHT, 3))
        
        for i in xrange(batch_size):
            if len(files) == 0:
                img_batch = img_batch[:i]
                break
            f = files.pop(0)
            path = DATA_PATH + 'test_stg2/' + f
            img = image.load_img(path)
            width, height = img.size
            img = img.resize((INPUT_WIDTH, INPUT_HEIGHT))
            img = image.img_to_array(img)
            img /= 255
            img_batch[i] = img
        
        yield img_batch
        

def visualize_prediction(img, index = None, true_box = None, pred_box = None, ax = None):
    if ax is None:
        _, ax = plt.subplots(1, 1, figsize=(12,8))
    ax.imshow(img)
    if index is not None:
        ax.set_title(index)
    height = img.shape[0]
    width = img.shape[1]
    
    if true_box is not None:
        x, y, w, h = true_box
        x = x * width
        y = y * height
        w = w * width
        h = h * height
        ax.add_patch(
        patches.Rectangle(
            (x, y), # x,y
            w, # width
            h, # height
            hatch='\\',
            fill=False,      # remove background
            color = 'r',
            linewidth = 2.5
                )
            )
    if pred_box is not None:
        x, y, w, h = pred_box
        x = x * width
        y = y * height
        w = w * width
        h = h * height
        ax.add_patch(
        patches.Rectangle(
            (x, y), # x,y
            w, # width
            h, # height
            hatch='-',
            fill=False,      # remove background
            color = 'k',
            linewidth = 2.5
                )
            )

def make_plot(data, nrow = 2, ncol = 2, index = None, true_box = None, pred_box = None, figsize = (15,8)):
    # Create grid
    _, ax = plt.subplots(nrow, ncol, figsize=figsize)
    
    idx = None
    tbox = None
    pbox = None
    # Generate indices of images to show
    for axi in np.ravel(ax):
        n = np.random.choice(len(data))
        img = data[n]
        if index is not None:
            idx = index[n]
        if true_box is not None:
            tbox = true_box[n]
        if pred_box is not None:
            pbox = pred_box[n]
        
        # Visualize it along with the box
        visualize_prediction(img, index = idx, true_box = tbox, pred_box = pbox, ax = axi)


# Crop all the fishes and save them as separate files.
def crop_all_fishes(id_test, y_pred):
    X_crop = []
    for i in xrange(len(id_test)):
        file_name = DATA_PATH + 'test_stg2/' + id_test[i]
        img = Image.open(file_name)
        
        width, height = img.size
        x, y, w, h = y_pred[i]
        left = int(max(x,0) * width)
        right = int(min((x + w), 1) * width)
        top = int(max(y,0) * height)
        bottom = int(min((y + h), 1) * height)
        img = img.crop((left, top, right, bottom))
        img = img.resize((224,224))
        img = image.img_to_array(img)
        img /= 255
        X_crop.append(img)
        if (i+1) in [k*len(id_test)/5 for k in xrange(1,6)]:
            print "Cropping fishes...{}% done!".format((i+2)*100/len(id_test))
    return np.array(X_crop)

def make_submission(id_test, y_pred):
    df1 = pd.DataFrame(id_test, columns = ["image"])
    df2 = pd.DataFrame(y_pred, columns = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'NoF'])
    df2_reorder = df2.reindex_axis(sorted(df2.columns), axis=1)
    submission = pd.concat([df1,df2_reorder],axis = 1)
    submission = submission.sort_values(by = "image")
    submission.to_csv(DATA_PATH + 'submission5.csv', header = True, index = False)
    return submission

## Load pretrained models

In [10]:
localizer = load_model(DATA_PATH + 'models/localizers/checkpoints/ResNet3SB.33-0.00.hdf5')
classifier = load_model(DATA_PATH + 'models/classifiers/classifier3.h5')

## Predict bounding boxes by localizer

In [11]:
test_folder = DATA_PATH + 'test_stg2/'
file_test = os.listdir(test_folder)
N_test = len(file_test)
batch_size = 30

steps = np.ceil(float(N_test)/batch_size)

pred_box = localizer.predict_generator(generator = test_data_generator(batch_size, file_test, INPUT_WIDTH, INPUT_HEIGHT),
                                       steps = steps,
                                       verbose = 1)

439/439 [==============================] - 188s   


In [ ]:
X_crop = crop_all_fishes(os.listdir(test_folder), pred_box)

Cropping fishes...20% done!
Cropping fishes...40% done!
Cropping fishes...60% done!
Cropping fishes...80% done!
Cropping fishes...100% done!


## Predict classes by classifier

In [ ]:
y_pred = classifier.predict(X_crop.astype('float32'), verbose=1)

  224/13153 [..............................] - ETA: 204s

## Make submission

In [9]:
files = os.listdir(test_folder)

print files[0:10]
for i, f in enumerate(files):
    if f.startswith("image"):
        files[i] = "test_stg2/" + f
print files[0:10]
submission = make_submission(files, y_pred)

['image_07987.jpg', 'image_00807.jpg', 'image_10284.jpg', 'image_02265.jpg', 'image_05811.jpg', 'image_01536.jpg', 'image_06540.jpg', 'image_00483.jpg', 'image_09769.jpg', 'image_10608.jpg']
['test_stg2/image_07987.jpg', 'test_stg2/image_00807.jpg', 'test_stg2/image_10284.jpg', 'test_stg2/image_02265.jpg', 'test_stg2/image_05811.jpg', 'test_stg2/image_01536.jpg', 'test_stg2/image_06540.jpg', 'test_stg2/image_00483.jpg', 'test_stg2/image_09769.jpg', 'test_stg2/image_10608.jpg']


In [ ]:
submission.head()